In [1]:
import os
import sys
import argparse
import pprint
import pdb
import time
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms

from datetime import datetime
from torch.utils.data.sampler import Sampler
from roi_data_layer.roidb import combined_roidb
from model.utils.config import cfg, cfg_from_file, cfg_from_list, get_output_dir


In [2]:
def parse_args():
    parser = argparse.ArgumentParser(description='Train FPN')
    parser.add_argument('--dataset', dest='dataset',
                        help='training dataset',
                        default='pascal_voc', type=str)
    parser.add_argument('--net', dest='net',
                        help='res101, res152, etc',
                        default='res101', type=str)
    parser.add_argument('--start_epoch', dest='start_epoch',
                        help='starting epoch',
                        default=1, type=int)
    parser.add_argument('--num_epochs', dest='num_epochs',
                        help='number of epochs to train',
                        default=20, type=int)
    parser.add_argument('--display_interval', dest='display_interval',
                        help='number of iterations to display',
                        default=100, type=int)
    parser.add_argument('--checkpoint_interval', dest='checkpoint_interval',
                        help='number of iterations to create checkpoint',
                        default=10000, type=int)
    
    parser.add_argument('--save_dir', dest='save_dir',
                        help='directory to save models',
                        default='', nargs=argparse.REMAINDER)

    args = parser.parse_args()
    return args
    

In [3]:
class sampler(Sampler):
  def __init__(self, train_size, batch_size):
    num_data = train_size
    self.num_per_batch = int(num_data / batch_size)
    self.batch_size = batch_size
    self.range = torch.arange(0,batch_size).view(1, batch_size).long()
    self.leftover_flag = False
    if num_data % batch_size:
      self.leftover = torch.arange(self.num_per_batch*batch_size, num_data).long()
      self.leftover_flag = True
  def __iter__(self):
    rand_num = torch.randperm(self.num_per_batch).view(-1,1) * self.batch_size
    self.rand_num = rand_num.expand(self.num_per_batch, self.batch_size) + self.range

    self.rand_num_view = self.rand_num.view(-1)

    if self.leftover_flag:
      self.rand_num_view = torch.cat((self.rand_num_view, self.leftover),0)

    return iter(self.rand_num_view)

  def __len__(self):
    return num_data

In [4]:
if __name__ == '__main__':
    # args = parse_args()

    # print('Called with args:')
    # print(args)
    imdb, roidb, ratio_list, ratio_index = combined_roidb('voc_2012_trainval')

Loaded dataset `voc_2012_trainval` for training
Set proposal method: gt
Appending horizontally-flipped training examples...
done
Preparing training data...
wrote gt roidb to /workspaces/FPN-with-Pytorch/datasets/cache/voc_2012_trainval_gt_roidb.pkl
done
before filtering, there are 0 images...
after filtering, there are 0 images...
